# Product Quantization

### Notebook written by Ryan Awad

Links:
- https://www.pinecone.io/learn/series/faiss/product-quantization/
- https://inria.hal.science/inria-00514462v2/document

In [147]:
!pip3 install numpy
from random import randint, uniform
import numpy as np
from math import log2, ceil

Defaulting to user installation because normal site-packages is not writeable


### We are defining 5 random 12 dimensional vectors

In [148]:
dim = 12
amount_of_vectors = 100
large_vectors = [[uniform(-100.0,100.0) for i in range(dim)] for i in range(amount_of_vectors)]
#large_vectors

### We are defining some variables for the algorithm

- `m`: dimensions of a compressed vector / how many sub-vectors will be created per vector
- `D_`: dimensions of a sub-vector

In [149]:
m = 4 # dimensions of a compressed vector

# ensure dim is divisable by m
assert dim % m == 0
# length of each subvector will be dim / m (D* in notation)
D_ = int(dim / m)

D_

3

In [150]:
# now create the subvectors
u_vectors = []
for x in large_vectors:
    u = [x[row:row+D_] for row in range(0, dim, D_)]
    u_vectors.append(u)

u_vectors

[[[-31.106114734363132, 91.39751427593416, 89.24741880387853],
  [37.40542222116093, 26.966670190659897, -76.65583421344778],
  [88.7552594849069, 4.763075457727311, 75.87582890354378],
  [17.92052682496407, 23.526217046605936, 31.977072091539526]],
 [[-36.081768873312605, 47.24080340920361, 91.71784866417903],
  [-77.58100049209769, -71.33120441371773, -34.2966796420836],
  [56.589939208515005, -37.75786218747221, -53.868267058956796],
  [-9.409127735976526, -57.90004105198423, -94.84326908335825]],
 [[-94.85146926031693, 95.81086692775386, -98.38449861442449],
  [-68.45973718948403, -31.78299998971464, -98.13587107301514],
  [18.98464648235752, -20.87855137972187, 1.8916475639642982],
  [-95.97324541287107, -80.8260323069897, -5.955085194932835]],
 [[-19.96951569631453, -67.10723747020145, -82.013966302846],
  [28.318478854516883, 60.47729205515381, 63.40556466850194],
  [4.185953119581853, -33.31970628299581, -20.546302718032365],
  [21.374097448933497, 54.62144229425613, 10.6849043

# Important
- `k`: total number of centroids generated
- `k_`: number of centroids to choose from per sub-vector

Notice how when you increase `k`, `k_` will increase proportionally. And as `k_` increases, the compressed vector size also increases, WHILE the distances between the regenerated vectors and the actual vectors decrease. In addition, as `k_` increases, so does the time needed to generate the centroids, as well as compressing a vector.

This is because as `k_` increases, each sub-vector gets to choose from a large amount of randomly generated centroids. Probablistically, a sub-vector will find a much better centroid than if `k_` was smaller which would give each sub-vector less options. This causes the average distance between regenerated vectors and actual vectors to decrease.

In addition, as `k_` increases, each sub-vector will have a larger amount of centroids to choose from, making the compression algorithm take longer

On the other hand, each sub-vector is converted into a centroid ID, as part of the compression algorithm. Since each sub-vector gets `k_` centroids to choose from, each ID will need to have allocated memory to store a value between 0 to `k_`-1. Therefore, as `k_` increases, the overall size of a compressed vector will increase. 

In [151]:
k = 2**10
assert k % m == 0
k_ = int(k/m)
print(f"{k=}, {k_=}")
print(f"Compressed vector size: {ceil(ceil(log2(k_))/8) * m} bytes")
print(f"Centroid map size: {ceil(k*D_*(32/8))} bytes")

k=1024, k_=256
Compressed vector size: 4 bytes
Centroid map size: 12288 bytes


### Randomly generating the centroids

In [152]:
c = []  # our overall list of reproduction values
for j in range(m):
    # each j represents a subvector (and therefore subquantizer) position
    c_j = []
    for i in range(k_):
        # each i represents a cluster/reproduction value position *inside* each subspace j
        c_ji = [uniform(-100.0,100.0) for _ in range(D_)]
        c_j.append(c_ji)  # add cluster centroid to subspace list
    # add subspace list of centroids to overall list
    c.append(c_j)
#c

In [153]:
def euclidean(v, u):
    distance = sum((x - y) ** 2 for x, y in zip(v, u)) ** .5
    return distance

'''
c_j is the SET of centroids for the specific sub-vector u_j

this function finds the nearest centroid to a sub-vector
'''
def nearest(c_j, u_j):
    distance = 9e+9
    for i in range(k_):
        new_dist = euclidean(c_j[i], u_j)
        if new_dist < distance:
            nearest_idx = i
            distance = new_dist
    return nearest_idx

### Compressing the vectors

In [154]:
compressed_vectors = [] # ids of the nearest centroid for each sub-vector
for u in u_vectors:
    curr_compressed_vec = []
    for j in range(m):
        i = nearest(c[j], u[j])
        curr_compressed_vec.append(i)
    compressed_vectors.append(curr_compressed_vec)

#print(f'old dimension: {len(x)}\ncompressed dimension: {len(ids)}')
compressed_vectors

[[75, 186, 7, 229],
 [5, 152, 217, 119],
 [17, 89, 109, 253],
 [49, 222, 255, 52],
 [182, 53, 92, 62],
 [147, 63, 234, 159],
 [6, 55, 101, 75],
 [79, 116, 182, 70],
 [104, 183, 110, 210],
 [76, 93, 79, 222],
 [157, 118, 6, 251],
 [113, 233, 47, 4],
 [216, 127, 213, 204],
 [238, 165, 141, 199],
 [219, 44, 128, 119],
 [250, 129, 22, 70],
 [44, 83, 46, 155],
 [33, 224, 143, 183],
 [38, 78, 131, 1],
 [122, 1, 219, 3],
 [219, 17, 199, 183],
 [180, 171, 130, 63],
 [250, 149, 107, 132],
 [183, 226, 36, 148],
 [213, 36, 212, 213],
 [174, 232, 119, 190],
 [10, 208, 125, 195],
 [200, 25, 255, 161],
 [235, 207, 46, 123],
 [96, 161, 253, 230],
 [26, 16, 168, 96],
 [194, 100, 96, 56],
 [113, 203, 82, 147],
 [224, 122, 188, 39],
 [34, 22, 174, 148],
 [229, 41, 114, 71],
 [33, 223, 175, 140],
 [76, 203, 206, 134],
 [75, 87, 74, 171],
 [211, 42, 117, 150],
 [204, 213, 175, 230],
 [41, 181, 13, 209],
 [72, 253, 14, 41],
 [60, 184, 112, 37],
 [158, 213, 40, 80],
 [18, 11, 190, 47],
 [86, 41, 22, 131],
 

### Defining an error function

This error function only cares about how much the distance of regenerated vectors <u>RELATIVE TO EACH OTHER</u> changed. This is because, in vector similarity search, the relative distance of vectors is the only thing that matters. They're actual position in the vector space means nothing if their distance relative to each other stays the same or similar. 

Computations
1. Computes the distance between all the uncompressed vectors, and the distance between the regenerated vectors
2. Computes the absolute different between the two distance arrays into an array
3. Returns the average absolute difference

### TODO:
- Since vectors can be of various size, the error can sometimes be really big, but at the same time not be signficant when compared relative to the vector's size.
    - The error should take the average absolute difference and make it relative to the size of the vectors

In [155]:
def get_transformation_err(regen, old) -> float:
    r_dists = []
    o_dists = []
    for i in range(len(regen)):
        for j in range(len(regen)):
            if i < j:
                r_dists.append(euclidean(regen[i], regen[j]))
    for i in range(len(old)):
        for j in range(len(old)):
            if i < j:
                o_dists.append(euclidean(old[i], old[j]))

    #print(r_dists)
    #print(o_dists)

    error = sum([abs(r - o) for r, o in zip(r_dists, o_dists)]) / len(r_dists)
    return error

### Regenerating the vectors from the compressed vectors

In [156]:
regen_vecs = []
for comp in compressed_vectors:
    curr_regen = []
    for j in range(m):
        c_ji = c[j][comp[j]] # take the nearest centroid to each sub-vector using their index i
        curr_regen.extend(c_ji)
    regen_vecs.append(curr_regen)

### Writing quick and rough naive $\mathcal{O}(n \log n)$ KNN algorithm for testing

In [157]:
def naive_knn(query_vector_id, k, vectors) -> list:
    neighbors: list[tuple[int, float]] = [] # i know i could use a priority queue but im lazy
    for i in range(len(vectors)):
        neighbors.append(
            (i+1, euclidean(vectors[query_vector_id-1], vectors[i]))
        )
    
    neighbors.sort(key=lambda x: x[1])
    return neighbors[:k]


### Results & Statistics
- Printing old and regenerated vectors
- The distance between regenerated and old vectors
- The average distance between regenerated and old vectors (not a great error metric as described in the cell above the error function definition)
- The output of the error function defined above
- The size of a compressed vector vs an old vector
- The compression $\%$

In [158]:
dists = []
for i in range(amount_of_vectors):
    dists.append(euclidean(regen_vecs[i], large_vectors[i]))

#print(f'Regenerated vectors from compressed vectors:\n{np.array(regen_vecs)}')
#print(f'Old vectors:\n{np.array(large_vectors)}')

print(f'Distances between decompressed and compress vectors:\n{dists}')
#print(f'Average distance between decompressed and compress vectors: {round(sum(dists)/amount_of_vectors, 2)}')
print(f'ERROR: {get_transformation_err(regen_vecs, large_vectors)}')

new_size = ceil(ceil(log2(k_))/8) * m
old_size = dim * ceil(32/8)

print(f"Compressed vector size: {new_size} bytes")
print(f"Old vector size: {old_size} bytes")
print(f"{round((1 - (new_size/old_size))*100, 2)}% compression")

Distances between decompressed and compress vectors:
[25.55121360642606, 44.626127876820775, 30.387961384832632, 47.81221420384642, 31.63606499141714, 40.614700261309835, 44.9230871688061, 50.920129453056816, 26.678602813897115, 44.52850052437109, 23.93748216385903, 43.45632796177762, 36.648545801143335, 61.28457034404383, 42.476907380939615, 37.46397954654534, 33.02804389099532, 28.81913250170618, 35.08736880043386, 37.63413902389762, 32.65816021615587, 51.94241834791902, 31.842614349000453, 44.44092148371029, 47.47205396714424, 29.36268220605889, 37.78299141186763, 36.53220796979337, 29.046576961110087, 38.63661689554907, 39.729480801987066, 41.9869179867871, 34.89506236163472, 36.702377699305366, 41.02165484376435, 39.9433448942169, 35.31762106125082, 51.521724041113906, 40.18398526917924, 41.008429170179674, 34.84989662026819, 26.74255404208468, 31.763203931034464, 36.37011182531777, 43.346223037881984, 44.372659663994135, 43.93644937156105, 38.844886972913535, 45.56201042861217, 4

### Testing similarity search with compressed vectors

In [159]:
test_query_vector_id = 2
neighbors = naive_knn(test_query_vector_id, 4, large_vectors)
print(neighbors)

neighbors = naive_knn(test_query_vector_id, 4, compressed_vectors)
print(neighbors)

[(2, 0.0), (20, 191.93293277036125), (37, 207.97281316863277), (29, 218.31036759093374)]
[(2, 0.0), (38, 89.37561188601732), (37, 89.61026726887941), (96, 97.48333190858835)]
